# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 8 2023 12:24PM,256943,10,0086364005,ISDIN Corporation,Released
1,Feb 8 2023 12:24PM,256943,10,0086364006,ISDIN Corporation,Released
2,Feb 8 2023 12:24PM,256943,10,0086364361,ISDIN Corporation,Released
3,Feb 8 2023 12:24PM,256943,10,0086364364,ISDIN Corporation,Released
4,Feb 8 2023 12:23PM,256942,19,ADV80011484,"AdvaGen Pharma, Ltd",Released
5,Feb 8 2023 12:23PM,256941,19,SOTAH0000901,"Nextsource Biotechnology, LLC",Released
6,Feb 8 2023 12:16PM,256940,10,Enova-11331,Emerginnova,Released
7,Feb 8 2023 12:16PM,256940,10,Enova-11332,Emerginnova,Released
8,Feb 8 2023 12:16PM,256940,10,Enova-11333,Emerginnova,Released
9,Feb 8 2023 12:07PM,256939,10,Clear-9132447,"ClearSpec, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
50,256939,Released,1
51,256940,Released,3
52,256941,Released,1
53,256942,Released,1
54,256943,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256939,NaN,NaN,1.0
256940,NaN,NaN,3.0
256941,NaN,NaN,1.0
256942,NaN,NaN,1.0
256943,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,25.0,20.0,7.0
256826,0.0,0.0,113.0
256827,0.0,13.0,3.0
256828,0.0,8.0,0.0
256857,0.0,0.0,11.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,25,20,7
256826,0,0,113
256827,0,13,3
256828,0,8,0
256857,0,0,11


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,25,20,7
1,256826,0,0,113
2,256827,0,13,3
3,256828,0,8,0
4,256857,0,0,11


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,25,20,7
1,256826,,,113
2,256827,,13,3
3,256828,,8,
4,256857,,,11


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 8 2023 12:24PM,256943,10,ISDIN Corporation
4,Feb 8 2023 12:23PM,256942,19,"AdvaGen Pharma, Ltd"
5,Feb 8 2023 12:23PM,256941,19,"Nextsource Biotechnology, LLC"
6,Feb 8 2023 12:16PM,256940,10,Emerginnova
9,Feb 8 2023 12:07PM,256939,10,"ClearSpec, LLC"
10,Feb 8 2023 12:07PM,256938,19,VITABIOTICS USA INC.
12,Feb 8 2023 11:55AM,256936,10,"Nextsource Biotechnology, LLC"
14,Feb 8 2023 11:54AM,256935,12,"Uniderm USA, Inc"
17,Feb 8 2023 11:48AM,256931,15,DXM Pharmaceutical Inc.
18,Feb 8 2023 11:41AM,256930,20,"HVL, LLC dba Atrium Innovations"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 8 2023 12:24PM,256943,10,ISDIN Corporation,,,4
1,Feb 8 2023 12:23PM,256942,19,"AdvaGen Pharma, Ltd",,,1
2,Feb 8 2023 12:23PM,256941,19,"Nextsource Biotechnology, LLC",,,1
3,Feb 8 2023 12:16PM,256940,10,Emerginnova,,,3
4,Feb 8 2023 12:07PM,256939,10,"ClearSpec, LLC",,,1
5,Feb 8 2023 12:07PM,256938,19,VITABIOTICS USA INC.,,,2
6,Feb 8 2023 11:55AM,256936,10,"Nextsource Biotechnology, LLC",,,2
7,Feb 8 2023 11:54AM,256935,12,"Uniderm USA, Inc",,2,1
8,Feb 8 2023 11:48AM,256931,15,DXM Pharmaceutical Inc.,,,1
9,Feb 8 2023 11:41AM,256930,20,"HVL, LLC dba Atrium Innovations",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 12:24PM,256943,10,ISDIN Corporation,4,,
1,Feb 8 2023 12:23PM,256942,19,"AdvaGen Pharma, Ltd",1,,
2,Feb 8 2023 12:23PM,256941,19,"Nextsource Biotechnology, LLC",1,,
3,Feb 8 2023 12:16PM,256940,10,Emerginnova,3,,
4,Feb 8 2023 12:07PM,256939,10,"ClearSpec, LLC",1,,
5,Feb 8 2023 12:07PM,256938,19,VITABIOTICS USA INC.,2,,
6,Feb 8 2023 11:55AM,256936,10,"Nextsource Biotechnology, LLC",2,,
7,Feb 8 2023 11:54AM,256935,12,"Uniderm USA, Inc",1,2,
8,Feb 8 2023 11:48AM,256931,15,DXM Pharmaceutical Inc.,1,,
9,Feb 8 2023 11:41AM,256930,20,"HVL, LLC dba Atrium Innovations",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 12:24PM,256943,10,ISDIN Corporation,4,,
1,Feb 8 2023 12:23PM,256942,19,"AdvaGen Pharma, Ltd",1,,
2,Feb 8 2023 12:23PM,256941,19,"Nextsource Biotechnology, LLC",1,,
3,Feb 8 2023 12:16PM,256940,10,Emerginnova,3,,
4,Feb 8 2023 12:07PM,256939,10,"ClearSpec, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 12:24PM,256943,10,ISDIN Corporation,4.0,NaN,NaN
1,Feb 8 2023 12:23PM,256942,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
2,Feb 8 2023 12:23PM,256941,19,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
3,Feb 8 2023 12:16PM,256940,10,Emerginnova,3.0,NaN,NaN
4,Feb 8 2023 12:07PM,256939,10,"ClearSpec, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 12:24PM,256943,10,ISDIN Corporation,4.0,0.0,0.0
1,Feb 8 2023 12:23PM,256942,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
2,Feb 8 2023 12:23PM,256941,19,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
3,Feb 8 2023 12:16PM,256940,10,Emerginnova,3.0,0.0,0.0
4,Feb 8 2023 12:07PM,256939,10,"ClearSpec, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5395068,65.0,61.0,3.0
12,256935,1.0,2.0,0.0
15,1284271,142.0,49.0,25.0
16,513784,1.0,1.0,0.0
19,3082866,27.0,23.0,3.0
20,256930,1.0,0.0,0.0
21,256870,1.0,0.0,0.0
22,256918,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5395068,65.0,61.0,3.0
1,12,256935,1.0,2.0,0.0
2,15,1284271,142.0,49.0,25.0
3,16,513784,1.0,1.0,0.0
4,19,3082866,27.0,23.0,3.0
5,20,256930,1.0,0.0,0.0
6,21,256870,1.0,0.0,0.0
7,22,256918,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,65.0,61.0,3.0
1,12,1.0,2.0,0.0
2,15,142.0,49.0,25.0
3,16,1.0,1.0,0.0
4,19,27.0,23.0,3.0
5,20,1.0,0.0,0.0
6,21,1.0,0.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,65.0
1,12,Released,1.0
2,15,Released,142.0
3,16,Released,1.0
4,19,Released,27.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,3.0,0.0,25.0,0.0,3.0,0.0,0.0,0.0
Executing,61.0,2.0,49.0,1.0,23.0,0.0,0.0,0.0
Released,65.0,1.0,142.0,1.0,27.0,1.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,3.0,0.0,25.0,0.0,3.0,0.0,0.0,0.0
1,Executing,61.0,2.0,49.0,1.0,23.0,0.0,0.0,0.0
2,Released,65.0,1.0,142.0,1.0,27.0,1.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,3.0,0.0,25.0,0.0,3.0,0.0,0.0,0.0
1,Executing,61.0,2.0,49.0,1.0,23.0,0.0,0.0,0.0
2,Released,65.0,1.0,142.0,1.0,27.0,1.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()